In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib, os, shutil
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
from scipy.stats import spearmanr
from statsmodels.sandbox.stats.multicomp import multipletests
from tqdm import tqdm_notebook
import leidenalg
import igraph as ig
import gseapy as gp
import pickle
from matplotlib import pyplot as plt
import pyranges as pr

In [ ]:
# def calc(df):
#     print('Calculating Correlation..')
#     temp=spearmanr(df.T)
#     corr=pd.DataFrame(temp[0],columns=list(df.index),index=list(df.index))
#     pval=pd.DataFrame(temp[1],columns=list(df.index),index=list(df.index))
#     print('Filtering the matrix Correlation..')
#     corr=corr.where(np.triu(np.ones(corr.shape)).astype(np.bool))
#     pval=pval.where(np.triu(np.ones(pval.shape)).astype(np.bool))
#     print('Making long table of Correlation..')
#     corr2=corr.unstack().reset_index(name='weight')
#     pval2=pval.unstack().reset_index(name='pval')
#     res=corr2.merge(pval2,on=['level_0','level_1'])
#     res=res[res['level_0'] != res['level_1']]
#     res=res.dropna()
#     res=res[['level_0','level_1','weight','pval']]
#     res['padj']=multipletests(res['pval'],method='fdr_bh')[1]
#     res=res[res.padj < 0.05].reset_index(drop=True)
#     print('Done!!')
#     return res

class Network_Analysis:
    def __init__(self,raw_data,nodes,respath,cat):
        self.res_path=respath
        self.cat=cat
        if os.path.isdir(self.res_path):
            pass
        else:
            os.mkdir(self.res_path)
        self.network_ori=self.__calc(raw_data)
        self.nodes=nodes

        print('Network Analysis')
        self.__net_analysis_combi()
    
    def __calc(self,df):
        print('Calculating Correlation..')
        temp=spearmanr(df.T)
        corr=pd.DataFrame(temp[0],columns=list(df.index),index=list(df.index))
        pval=pd.DataFrame(temp[1],columns=list(df.index),index=list(df.index))
        print('Filtering the matrix Correlation..')
        corr=corr.where(np.triu(np.ones(corr.shape)).astype(np.bool))
        pval=pval.where(np.triu(np.ones(pval.shape)).astype(np.bool))
        print('Making long table of Correlation..')
        corr2=corr.unstack().reset_index(name='weight')
        pval2=pval.unstack().reset_index(name='pval')
        res=corr2.merge(pval2,on=['level_0','level_1'])
        res=res[res['level_0'] != res['level_1']]
        res=res.dropna()
        res=res[['level_0','level_1','weight','pval']]
        res['category'] = self.cat
        res['padj']=multipletests(res['pval'],method='fdr_bh')[1]
        res=res[res.padj < 0.05].reset_index(drop=True)
        res.columns=['source','target','correlation','pvalue','category','padj']
        res=res[['source','target','category','correlation','pvalue','padj']]
        res.to_csv('%s/%d_edges.txt' % (self.res_path,self.cat),sep='\t',index=False)
        print('Done!!')
        return res
    
    def __net_analysis_combi(self):
        print('Loading The Network...')
        temp=self.network_ori
        g= ig.Graph.TupleList(zip(temp['source'],temp['target'],temp['correlation']),weights=True)
        G_pos = g.subgraph_edges(g.es.select(weight_gt = 0), delete_vertices=False)
        G_neg = g.subgraph_edges(g.es.select(weight_lt = 0), delete_vertices=False)
        G_neg.es['weight'] = [-w for w in G_neg.es['weight']]
        part_pos = leidenalg.ModularityVertexPartition(G_pos, weights='weight')
        part_neg = leidenalg.ModularityVertexPartition(G_neg, weights='weight');
        optimiser = leidenalg.Optimiser()
        diff = optimiser.optimise_partition_multiplex([part_pos, part_neg],layer_weights=[1,-1], n_iterations=-1)
        self.clustering_combi=pd.DataFrame(pd.Series(part_pos.membership+part_neg.membership,index=G_pos.vs['name']+G_neg.vs['name'])).reset_index().drop_duplicates().set_index('index')[0]
        print('Cluster Analysis...')
        self.modularity_combi=diff
        open('%s/%d_modularity.txt' % (self.res_path,self.cat),'w').write(str(diff))
        self.nodes['cluster'] = self.clustering_combi.reindex(self.nodes.index).tolist()
        self.nodes.to_csv('%s/%d_nodes.txt' % (self.res_path,self.cat),sep='\t')
    
    def save_network(self):
        pickle_out = open('%s/%d_network_object.pkl' % (self.res_path,self.cat),"wb")
        pickle.dump(self, pickle_out)
        pickle_out.close()

In [ ]:
# meta=pd.read_csv('../Data/GTEX/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt',sep='\t',index_col=0)[['SMTSD']]
# meta2=pd.read_csv('../Data/GTEX/GTEx_Analysis_v8_Annotations_SubjectPhenotypesDS.txt',sep='\t',index_col=0)[['SEX','AGE']]
# new_meta=[]
# for i in meta.index:
#     temp='-'.join(i.split('-')[0:2])
#     new_meta.append([meta2.loc[temp]['SEX'],meta2.loc[temp]['AGE']])
# new_meta=pd.DataFrame(new_meta,index=meta.index,columns=['SEX','AGE'])
# meta=pd.concat([meta,new_meta],1)
# data=pd.read_csv('../Data/GTEX/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz', sep='\t',skiprows=2,nrows=2,index_col=0)
# # meta=pd.read_csv('../Data/GTEX/metadata_final.txt',sep='\t',index_col=0)
# meta.reindex(data.columns[1:]).to_csv('../Data/GTEX/metadata_final.txt',sep='\t')

# pc=pr.read_gtf('../Data/GTEX/gencode.v26.GRCh38.genes.gtf').df
# pc=pc[pc['gene_type'] == 'protein_coding'][['gene_id','gene_name']].drop_duplicates()
# pc['ensembl']=[i.split('.')[0] for i in pc['gene_id']]
# protein_atlas=pd.read_csv('../../MainLibrary/proteinatlas.tsv',sep='\t',index_col='Ensembl')[['Gene','Gene synonym','Uniprot']]
# pc=pc.merge(protein_atlas,right_on='Ensembl',left_on='ensembl')
# pc.set_index('gene_id').to_csv('../Data/GTEX/protein_coding.txt',sep='\t')

In [ ]:
meta=pd.read_csv('../Data/GTEX/metadata_final.txt',sep='\t',index_col=0)
pc=pd.read_csv('../Data/GTEX/protein_coding.txt',sep='\t',index_col=0)
cat=pd.DataFrame(meta['SMTSD'].unique(),columns=['name_category']).reset_index()
cat.columns=['id_category','name_category']
cat['type_category'] = 'Normal Tissue'
cat=cat.set_index('id_category')
cat.to_csv('../Results/GTEx/category.txt',sep='\t')

In [ ]:
for i in cat.index[0:2]:
    temp=['Name']+list(meta[meta['SMTSD'] == cat.loc[i]['name_category']].index)
    tpm=pd.read_csv('../Data/GTEX/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz', sep='\t',skiprows=2,index_col=0,usecols=temp).reindex(set(pc.index)).dropna(how='all')
    tpm=tpm[tpm.mean(1)>1]
    nodes=pc[['Gene','ensembl']].reindex(tpm.index)
    nodes.columns=['symbol','info1']
    mean=tpm.mean(1)
    sd=tpm.std(1)
    mins=tpm.min(1)
    maxs=tpm.max(1)
    ran=['%.1f - %.1f' % (mins.loc[k],maxs.loc[k]) for k in mins.index]
    mean=['%.1f (+/- %.1f)' % (mean.loc[k],sd.loc[k]) for k in mins.index]
    nodes['info2'] = mean
    nodes['info3'] =ran
    nodes['location'] = 'GENE'
    nodes['category'] = i
    print('StartingNet')
    k=Network_Analysis(raw_data=tpm,nodes=nodes,respath='../Results/GTEx/',cat=i)
    print('SavingNet')
    k.save_network()

In [ ]:
import pandas as pd
import numpy as np
import os, sys, leidenalg, pickle
from scipy.stats import spearmanr
from statsmodels.sandbox.stats.multicomp import multipletests
import igraph as ig

class Network_Analysis:
    def __init__(self,raw_data,nodes,respath,cat):
        self.res_path=respath
        self.cat=cat
        if os.path.isdir(self.res_path):
            pass
        else:
            os.mkdir(self.res_path)
        self.network_ori=self.__calc(raw_data)
        self.nodes=nodes

        print('Network Analysis')
        self.__net_analysis_combi()
    
    def __calc(self,df):
        print('Calculating Correlation..')
        temp=spearmanr(df.T)
        corr=pd.DataFrame(temp[0],columns=list(df.index),index=list(df.index))
        pval=pd.DataFrame(temp[1],columns=list(df.index),index=list(df.index))
        print('Filtering the matrix Correlation..')
        corr=corr.where(np.triu(np.ones(corr.shape)).astype(np.bool))
        pval=pval.where(np.triu(np.ones(pval.shape)).astype(np.bool))
        print('Making long table of Correlation..')
        corr2=corr.unstack().reset_index(name='weight')
        pval2=pval.unstack().reset_index(name='pval')
        res=corr2.merge(pval2,on=['level_0','level_1'])
        res=res[res['level_0'] != res['level_1']]
        res=res.dropna()
        res=res[['level_0','level_1','weight','pval']]
        res['category'] = self.cat
        res['padj']=multipletests(res['pval'],method='fdr_bh')[1]
        res=res[res.padj < 0.05].reset_index(drop=True)
        res.columns=['source','target','correlation','pvalue','category','padj']
        res=res[['source','target','category','correlation','pvalue','padj']]
        res.to_csv('%s/%d_edges.txt' % (self.res_path,self.cat),sep='\t',index=False)
        print('Done!!')
        return res
    
    def __net_analysis_combi(self):
        print('Loading The Network...')
        temp=self.network_ori
        g= ig.Graph.TupleList(zip(temp['source'],temp['target'],temp['correlation']),weights=True)
        G_pos = g.subgraph_edges(g.es.select(weight_gt = 0), delete_vertices=False)
        G_neg = g.subgraph_edges(g.es.select(weight_lt = 0), delete_vertices=False)
        G_neg.es['weight'] = [-w for w in G_neg.es['weight']]
        part_pos = leidenalg.ModularityVertexPartition(G_pos, weights='weight')
        part_neg = leidenalg.ModularityVertexPartition(G_neg, weights='weight');
        optimiser = leidenalg.Optimiser()
        diff = optimiser.optimise_partition_multiplex([part_pos, part_neg],layer_weights=[1,-1], n_iterations=-1)
        self.clustering_combi=pd.DataFrame(pd.Series(part_pos.membership+part_neg.membership,index=G_pos.vs['name']+G_neg.vs['name'])).reset_index().drop_duplicates().set_index('index')[0]
        print('Cluster Analysis...')
        self.modularity_combi=diff
        open('%s/%d_modularity.txt' % (self.res_path,self.cat),'w').write(str(diff))
        self.nodes['cluster'] = self.clustering_combi.reindex(self.nodes.index).tolist()
        self.nodes.to_csv('%s/%d_nodes.txt' % (self.res_path,self.cat),sep='\t')
    
    def save_network(self):
        pickle_out = open('%s/%d_network_object.pkl' % (self.res_path,self.cat),"wb")
        pickle.dump(self, pickle_out)
        pickle_out.close()

meta=pd.read_csv('../Data/GTEX/metadata_final.txt',sep='\t',index_col=0)
pc=pd.read_csv('../Data/GTEX/protein_coding.txt',sep='\t',index_col=0)
cat=pd.read_csv('../Data/GTEX/category.txt',sep='\t',index_col=0)

i = int(sys.argv[1])

temp=['Name']+list(meta[meta['SMTSD'] == cat.loc[i]['name_category']].index)
tpm=pd.read_csv('../Data/GTEX/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz', sep='\t',skiprows=2,index_col=0,usecols=temp).reindex(set(pc.index)).dropna(how='all')
tpm=tpm[tpm.mean(1)>1]
nodes=pc[['Gene','ensembl']].reindex(tpm.index)
nodes.columns=['symbol','info1']
mean=tpm.mean(1)
sd=tpm.std(1)
mins=tpm.min(1)
maxs=tpm.max(1)
ran=['%.1f - %.1f' % (mins.loc[k],maxs.loc[k]) for k in mins.index]
mean=['%.1f (+/- %.1f)' % (mean.loc[k],sd.loc[k]) for k in mins.index]
nodes['info2'] = mean
nodes['info3'] =ran
nodes['location'] = 'GENE'
nodes['category'] = i
print('StartingNet')
k=Network_Analysis(raw_data=tpm,nodes=nodes,respath='../Results/GTEx/',cat=i)
# print('SavingNet')
# k.save_network()
print('Done %d' % i)

In [ ]:
(i+1) % 20

In [ ]:
x=0
for i in cat.index:
    print('python GTEX.py %d &' % i)
    if (i+1) % 3 == 0:
        print('wait')
print('wait') 

In [9]:
cat=pd.read_csv('../Data/GTEX/category.txt',sep='\t',index_col=0)
x=[]
for i in cat.index:
    temp=float(open('../Results/GTEx/modularity/%s_modularity.txt' % i, 'r').read())
    x.append(temp)
cat['modularity']=x
cat.to_csv('../Results/GTEx/category.txt',sep='\t')

In [7]:
temp

0.8211475790129228